Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 '10X_PBMC']

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [ ]:
# df = pd.read_pickle(f"{path}output/pickle_results/real_data/real_data_nb_genes.pkl")

In [11]:
# df = df[~df["nb_genes"].isin(["random_half","all"])]

In [13]:
df = pd.DataFrame()
dropout = 0.9
lr = 0.4
layers = [200, 40, 60]
# layers = [50]
temperature = 0.07
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())


   
    for nb_genes in ["random_half","all", 300, 400 ,200, 500, 1000, 1500, 3000]:
        for run in range(3):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            
            data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            cluster_number = np.unique(Y).shape[0]
            if nb_genes == "all":
                X = train.preprocess(X, nb_genes=X.shape[1])
            elif nb_genes == "random_half":
                X = train.preprocess(X, nb_genes=X.shape[1])
                X, X_test, Y, y_test = train_test_split(X, Y, train_size=0.5, random_state=run)
            else:
                X = train.preprocess(X, nb_genes=nb_genes)


            dresults = train.run(X,
                                 cluster_number,
                                 dataset,
                                 Y=Y,
                                 nb_epochs=30,
                                 lr=lr,
                                 temperature=temperature,
                                 dropout=dropout,
                                 evaluate=False,
                                 n_ensemble=1,
                                 layers=layers,
                                 save_to=f"{path}output/real_data/{run}/",
                                 save_pred = False)
            dresults["temperature"] = temperature
            dresults["dropout"] = dropout
            dresults["nb_genes"] = nb_genes
            dresults["layers"] = str(layers)
            dresults["run"] = run
            print(f".", end = "")
            print(f"# {temperature}, {dropout}, {lr}, {layers}", 
                  dresults.get('COMBINED_kmeans_ari', ""),
                  dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
                  dresults.get('leiden_ari_0', ""))
            df = df.append(dresults, ignore_index=True)

            df.to_pickle(f"{path}output/pickle_results/real_data/real_data_nb_genes.pkl")

>>>>> Data Quake_Smart-seq2_Trachea
SCZI  0.8291128219663276
(1350, 23341) (1350, 23341) keeping 23341 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.40245008668852794 0.1869397660639406
(1350, 23341) (1350, 23341) keeping 23341 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.34783037588608207 0.1939339263597727
(1350, 23341) (1350, 23341) keeping 23341 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.42174005183914615 0.17939127013582096
(1350, 23341) (1350, 23341) keeping 23341 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.4344659840864525 0.1434685488127364
(1350, 23341) (1350, 23341) keeping 23341 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.3989805203367941 0.16633778636859076
(1350, 23341) (1350, 23341) keeping 23341 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.2532389508031676 0.13613806288460348
(1350, 23341) (1350, 23341) keeping 300 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.753390602471723 0.1606292473892888
(1350, 23341) (1350, 23341) keeping 300 genes
.# 0.07, 0.9, 1e-05, [200, 40

.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6427443897638473 0.05505715576479915
(9552, 23341) (9552, 23341) keeping 200 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6694814315237587 0.04682659892732993
(9552, 23341) (9552, 23341) keeping 200 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.46286390251257187 0.0521018968841102
(9552, 23341) (9552, 23341) keeping 500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9179554666263199 0.10285435216195174
(9552, 23341) (9552, 23341) keeping 500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9058783786676112 0.093658663050477
(9552, 23341) (9552, 23341) keeping 500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8935893555591153 0.08174096312730321
(9552, 23341) (9552, 23341) keeping 1000 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5045753062680428 0.09985669833056235
(9552, 23341) (9552, 23341) keeping 1000 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5015013394834423 0.11603611244789135
(9552, 23341) (9552, 23341) keeping 1000 genes
.# 0.07, 0.9, 1e-05, [2

(2717, 24175) (2717, 24175) keeping 3000 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7768415737065703 0.328667509595966
(2717, 24175) (2717, 24175) keeping 3000 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7596843746780378 0.31249305015771195
>>>>> Data Adam
SCZI  0.8634561030635544
(3660, 23797) (3660, 23797) keeping 23797 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5559608319715287 0.5034709734922489
(3660, 23797) (3660, 23797) keeping 23797 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5079304295918886 0.5067075626944747
(3660, 23797) (3660, 23797) keeping 23797 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6420631022291784 0.5058477854410914
(3660, 23797) (3660, 23797) keeping 23797 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6499569896008229 0.4888624120303478
(3660, 23797) (3660, 23797) keeping 23797 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6422989555841261 0.5081947753007213
(3660, 23797) (3660, 23797) keeping 23797 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6401543797999

.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6858216755004962 0.3743790997871489
(1676, 23341) (1676, 23341) keeping 400 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7433247789797301 0.33267977011372857
(1676, 23341) (1676, 23341) keeping 200 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.4206914335558029 0.24080917206467672
(1676, 23341) (1676, 23341) keeping 200 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.4525622047057246 0.23434541114861515
(1676, 23341) (1676, 23341) keeping 200 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.4496387842675852 0.21201657373503255
(1676, 23341) (1676, 23341) keeping 500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.613425346037264 0.34456964576177795
(1676, 23341) (1676, 23341) keeping 500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7743684865889761 0.3442837723565648
(1676, 23341) (1676, 23341) keeping 500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7315983320715403 0.36041955943000686
(1676, 23341) (1676, 23341) keeping 1000 genes
.# 0.07, 0.9, 1e-05, [200,

(4186, 13488) (4186, 13488) keeping 1500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.38025388883919115 0.29431403823916513
(4186, 13488) (4186, 13488) keeping 3000 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5057093581747143 0.43846957482750226
(4186, 13488) (4186, 13488) keeping 3000 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.4735974990242236 0.4601629967689451
(4186, 13488) (4186, 13488) keeping 3000 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.4710630490290705 0.40993753849133285
>>>>> Data mouse_bladder_cell
SCZI  0.4148792388055975
(2746, 20670) (2746, 20670) keeping 20670 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.3422610886553683 0.3660779784646021
(2746, 20670) (2746, 20670) keeping 20670 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.37956148622122193 0.3631355074442469
(2746, 20670) (2746, 20670) keeping 20670 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.3621253503020792 0.4087136902609795
(2746, 20670) (2746, 20670) keeping 20670 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60] 

.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9121397448071397 0.32668260813443545
(1090, 23341) (1090, 23341) keeping 400 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9607439279909008 0.30341557083315585
(1090, 23341) (1090, 23341) keeping 400 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8933703620816044 0.2881703075209825
(1090, 23341) (1090, 23341) keeping 400 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9673862398165582 0.3089212193112497
(1090, 23341) (1090, 23341) keeping 200 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6099539585166641 0.16883476431260314
(1090, 23341) (1090, 23341) keeping 200 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5810739283871994 0.17973701893297353
(1090, 23341) (1090, 23341) keeping 200 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5935031174846305 0.17202221345939975
(1090, 23341) (1090, 23341) keeping 500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9722957002915682 0.35379376911093435
(1090, 23341) (1090, 23341) keeping 500 genes
.# 0.07, 0.9, 1e-05, [200,

(4271, 16653) (4271, 16653) keeping 1500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7712082388250349 0.42995238583468515
(4271, 16653) (4271, 16653) keeping 1500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7597491176275699 0.4376242969079027
(4271, 16653) (4271, 16653) keeping 1500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7690687005469913 0.4558098491223015
(4271, 16653) (4271, 16653) keeping 3000 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7921875889774744 0.5392779113361134
(4271, 16653) (4271, 16653) keeping 3000 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7672235175533728 0.4896667280787936
(4271, 16653) (4271, 16653) keeping 3000 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7859777489407299 0.5017982554074962


In [14]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")
sczi = sczi[sczi["dataset"].isin(files)]
sczi.mean()

ARI       0.745684
NMI       0.777119
sil       0.070277
run       1.000000
time    195.854597
dtype: float64

In [15]:
df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_nb_genes.pkl")
df = df[df["dataset"].isin(files)]
df.groupby(["temperature", "layers", "dropout", "nb_genes"])["kmeans_ari_0"].mean().unstack(["layers", "nb_genes"]).T

temperature                    0.07
dropout                         0.9
layers        nb_genes             
[200, 40, 60] 200          0.504284
              300          0.633631
              400          0.704243
              500          0.736921
              1000         0.701773
              1500         0.680087
              3000         0.655036
              all          0.500469
              random_half  0.495459

In [ ]:
# df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "nb_genes"])["kmeans_ari_0"].mean().unstack(["layers", "nb_genes"]).max()

In [ ]:
# r = df[(df["layers"]=="[40]")
#        &(df["temperature"]==0.01)
#        &(df["lr"]==1e-5)
#        &(df["dropout"]==0.9)
#       ]
r = df[(df["layers"]=="[200, 100, 30, 30]")
       &(df["temperature"]==0.07)
       &(df["lr"]==1e-5)
       &(df["dropout"]==0.9)
      ]


r.mean()

In [ ]:
comb = pd.merge(r, sczi, on=["dataset", "run"])[[
    "dataset", "kmeans_ari_0", "ARI", "kmeans_nmi_0", "NMI"
]].rename(columns = {"kmeans_ari_0": "contrative-sc", "ARI": "sczi"})
comb

In [ ]:
comb =pd.melt(comb, id_vars=["dataset"], value_vars=["contrative-sc", "sczi"])

In [ ]:
comb.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot( data = comb, y="value", x = "variable")
plt.xticks(rotation = 90)

In [ ]:
sns.barplot(x = "dataset", data = comb, y="value", hue = "variable")
plt.xticks(rotation = 90)

In [ ]:
df.groupby("dataset").mean()

# Importance of input size

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_genes", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    for run in range(2):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        for nb_genes in [100, 200, 500, 1000, 1500, 2000, 5000, 8000]:

            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
            print(f"Perc 0 {perc_0}")
            cluster_number = np.unique(Y).shape[0]

            X = train.preprocess(X, nb_genes=nb_genes)
            nb_zeros = int(0.8 * nb_genes)
            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=100,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/inputs/{dataset}_{nb_genes}/")

            #         df.loc[df.shape[0]] = [
            #                 dataset, perc_0, nb_genes, 'kmeans_representation_0',dresults['kmeans_representation_0']
            #             ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, nb_genes, 'leiden_representation_0',
                dresults['leiden_representation_0'], run]

#             pxt = PCA(2).fit_transform(X)
#             dresults["original"] = utils.evaluate(X, Y, cluster_number)[1]
#             dresults["pca"] = utils.evaluate(pxt, Y, cluster_number)[1]
            print(dresults)
    df.to_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df.groupby(["nb_genes"]).mean()

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_genes": "Nb input genes"})

In [ ]:
df["dataset"].unique()


In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size.pdf", bbox_inches='tight')

In [ ]:
datasets = ['10X PBMC (2100)',
       'Mouse ES\nCell (2100)', 'Worm Neuron\nCell (2100)',
       'Mouse Bladder\n Cell (2100)']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size_2100.pdf", bbox_inches='tight')

# Importance of the number of epochs

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_epochs", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    nb_genes = 1500
    for epochs in [5, 50, 100, 300]:

        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
        print(f"Perc 0 {perc_0}")
        cluster_number = np.unique(Y).shape[0]

        X = train.preprocess(X, nb_genes=nb_genes)
        nb_zeros = int(0.8 * nb_genes)
        for run in range(2):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=epochs,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/epochs/{dataset}_{epochs}/")

            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'kmeans_representation_0',
                dresults['kmeans_representation_0'], run
            ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'leiden_representation_0',
                dresults['leiden_representation_0'], run
            ]

            print(dresults)
            df.to_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_epochs": "Nb epochs"})

In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(7, 3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb epochs",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title ="Number of epochs",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_nb_epochs.pdf", bbox_inches='tight')